In [44]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [45]:
df = getDF('/home/nick/RQ-VAE-Recommender/dataset/beauty/raw/reviews_Beauty_5.json.gz')

In [46]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1.0,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3.0,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4.0,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2.0,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3.0,It's okay.,1382140800,"10 19, 2013"


In [47]:
import pandas as pd
import gzip
from tqdm import tqdm

def parse(path):
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)

def getDF_cat(path, cat):
    i = 0
    df = {}
    
    # 獲取總行數的估算（這裡假設每行代表一個物件）
    total_lines = sum(1 for _ in gzip.open(path, 'rb'))
    
    # 使用 tqdm 顯示進度條
    for d in tqdm(parse(path), total=total_lines, desc="Processing items"):
        # 檢查是否有 categories 欄位
        if 'categories' not in d:
            continue

        # 檢查 categories 中是否包含指定的類別 cat
        if any(cat == category_list[0] for category_list in d['categories']):
            df[i] = d
            i += 1

    return pd.DataFrame.from_dict(df, orient='index')


In [48]:
df_meta = getDF_cat('/home/nick/RQ-VAE-Recommender/dataset/beauty/raw/metadata.json.gz', 'Beauty')

Processing items: 100%|██████████| 9430088/9430088 [06:43<00:00, 23342.33it/s]


In [49]:
df_meta.to_pickle('/home/nick/RQ-VAE-Recommender/dataset/beauty/processed/metadata.pkl')

In [50]:
df_meta = pd.read_pickle('/home/nick/RQ-VAE-Recommender/dataset/beauty/processed/metadata.pkl')

In [51]:
len(df_meta)

259204

In [53]:
df_meta.head()

,asin,description,title,imUrl,salesRank,categories,price,related,brand
0,0205616461,"As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,{'Health & Personal Care': 461765},"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
1,0558925278,Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,{'Beauty': 402875},"[[Beauty, Tools & Accessories, Makeup Brushes ...",NaN,NaN,NaN
2,0733001998,"From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,{'Beauty': 540255},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",NaN,NaN,NaN
3,0737104473,Limited edition Hello Kitty Lipstick featuring...,Hello Kitty Lustre Lipstick (See sellers comme...,http://ecx.images-amazon.com/images/I/31u6Hrzk...,{'Beauty': 931125},"[[Beauty, Makeup, Lips, Lipstick]]",NaN,NaN,NaN
4,0762451459,"The mermaid is an elusive (okay, mythical) cre...",Stephanie Johnson Mermaid Round Snap Mirror,http://ecx.images-amazon.com/images/I/41y2%2BF...,NaN,"[[Beauty, Tools & Accessories, Mirrors, Makeup...",19.98,NaN,NaN


In [82]:

def create_sentence(row):
    # 初始化句子列表
    parts = []
    
    # 檢查每個欄位並組合句子
    if pd.notna(row['title']):
        parts.append(f"Title: {row['title']}")
    if pd.notna(row['brand']):
        parts.append(f"Brand: {row['brand']}")
    if isinstance(row['categories'], list) and row['categories']:
        categories_str = ''
        for idx, sublist in enumerate(row['categories']):
            categories_str += ', '.join(cat for cat in sublist)
            if idx < len(row['categories']) - 1:
                categories_str += ' | '
        parts.append(f"Categories: {categories_str}")        
    if pd.notna(row['salesRank']):
        rank_str = '' 
        for key, value in row['salesRank'].items():
            rank_str += f"{key} {value}"
            if key != list(row['salesRank'].keys())[-1]:
                rank_str += ' | '
        parts.append(f"Sales Rank: {rank_str}")
    
    # 組合句子並返回
    return '. '.join(parts) + '.' if parts else 'No information available.'
df_meta['sentence'] = df_meta.apply(create_sentence, axis=1)

In [86]:
df_meta.to_pickle('/home/nick/RQ-VAE-Recommender/dataset/beauty/processed/metadata_sentence.pkl')